In [3]:
import openpyxl

In [4]:
# open the excel file
file = "test02.xlsx"

# load the workbook
workbook = openpyxl.load_workbook(file)

# load the worksheet
worksheet = workbook['Current roster']

# a function for printing column names
def print_colnames():
    col_names = []
    for col in worksheet.iter_cols(1, worksheet.max_column):
        col_names.append(col[0].value)
    print(col_names)
    
print_colnames()

['Room', 'Type', 'Email', 'First name', 'Last name', 'Student ID', 'Gender', 'Class', 'Move-in date', 'Move-out date', 'Roommate preference', 'Status']


In [5]:
# a function that selects a specific column by name
def COL_select(column):
    Col_Names = {}
    Current = 0 
    for COL in worksheet.iter_cols(1, worksheet.max_column):
        Col_Names[COL[0].value] = Current
        Current += 1
    Col_list = []
    for row_cells in worksheet.iter_rows(1, worksheet.max_row):
        Col_list.append(row_cells[Col_Names[column]])
    return Col_list

In [6]:
# a function that finds a tenant unit by first name
def find_by_fn(fname):
    for row_cells in worksheet.iter_rows(1, worksheet.max_row):
        if row_cells[Col_Names['First name']].value == fname:
            print(row_cells[Col_Names['Room']].value)

In [7]:
# initiate a class which stores different attributes of a tenant 
class Tenant: 

    def __init__(self, first_name, last_name, student_id, email, gender, class_yr, room_type, move_in, move_out, roommate_pref, unit = "unassigned"):
        self.fn = first_name
        self.ln = last_name
        self.id = student_id
        self.em = email
        self.gn = gender
        self.yr = class_yr
        self.rt = room_type
        self.mi = move_in
        self.mo = move_out
        self.rm = roommate_pref
        self.un = unit
    

In [8]:
count = 0 
for i in COL_select("Class"):
    if i.value == "M21":
        count += 1
print(count)

79


In [9]:
def position_tenant(cell_row):
    COL_select("Email")[cell_row-1].value = tenant.em
    COL_select("First name")[cell_row-1].value = tenant.fn
    COL_select("Last name")[cell_row-1].value = tenant.ln
    COL_select("Student ID")[cell_row-1].value = tenant.id
    COL_select("Gender")[cell_row-1].value = tenant.gn
    COL_select("Class")[cell_row-1].value = tenant.yr
    COL_select("Move-in date")[cell_row-1].value = tenant.mi
    COL_select("Move-out date")[cell_row-1].value = tenant.mo 
    COL_select("Roommate preference")[cell_row-1].value = tenant.rm
    tenant.un = COL_select("Room")[cell_row-1].value

def new_tenant():
    first_name = input("First name: ")
    last_name = input("Last name: ")
    student_id = input("Student ID: ")
    email = input("Student email address: ")
    gender = input("Gender: ")
    class_yr = input("Class: ")
    room_type = input("Room type: ")
    move_in = input("Move-in date (MM/DD/YY): ")
    move_out = input("Move-out date (MM/DD/YY): ")
    roommate_pref = input("Roommate preference ID: ")
    unit = input("Assigned unit: ")

    if unit == "":
        tenant = Tenant(first_name, last_name, student_id, email, gender, class_yr, room_type, move_in, move_out, roommate_pref)
    else:
        tenant = Tenant(first_name, last_name, student_id, email, gender, class_yr, room_type, move_in, move_out, roommate_pref, unit)

    return tenant

# place the tenant if they have a roommate preference 
def place_roommate(roommate_id):
    for cell in COL_select("Student ID"):
        if cell.value == int(roommate_id):
            room = COL_select("Room")[cell.row-1].value[:-1]
            row = cell.row
        else:
            raise NotImplementedError("Roommate mismatch")
    

    
    #         if room[-1] == "A":
    #             room = COL_select("Room")[cell.row-1].value[:-1] + "B"
    #         else:
    #             room = COL_select("Room")[cell.row-1].value[:-1] + "A"
    #         break
    # for room_1 in COL_select("Room"):
    #     if room_1.value == room and COL_select("Email")[room_1.row-1] == None:
    #         position_tenant(room_1.row)
    #         break
    #     else: 
    #         raise NotImplementedError("Assignment failed due to roommate preference")

# check a tenant's gender match with their roommate 
## >>>> check the room name and the tenant status ++ DONE!!!
def gender_check(cell):
    room = COL_select("Room")[cell.row-1].value[:-1]
    roommate_gen = ""
    roommates = []
    for cell in COL_select("Room"):
        if room in str(cell.value):
            roommates.append(cell)
    for roommate in roommates:
        row = roommate.row
        if COL_select("Status")[roommate.row-1].value == "Current Tenant":
            roommate_gen = COL_select("Gender")[roommate.row-1].value
    
    if tenant.gn == roommate_gen or roommate_gen == None:
        return True
    else:
        return False

# a function that assigns a new tenant into a room and returns the unit number
def new_tenant_assign(tenant):

    # if the room is assigned already, it will just place the tenant
    if tenant.un != "unassigned":
        for cell in COL_select("Room"):
            if cell.value == tenant.un:
                position_tenant(cell.row)
                break
            else: 
                raise NotImplementedError("No such room exists")
    else:
        for cell in COL_select("Type"):
            # first, check for an availability via room type
            if (cell.value == tenant.rt and COL_select("Email")[cell.row-1].value == None) and (tenant.rm == "" and gender_check(cell)):
                position_tenant(cell.row)
                break
            # second, check roommate preference 
            elif (cell.value == tenant.rt and COL_select("Email")[cell.row-1].value == None) and (tenant.rm !=  ""):
                place_roommate(tenant.rm)
                break
            else:
                continue
                    
                    
                    # fourth, optimize the dates 
        # checks if the 
        if tenant.un == "unassigned":
            print("The tenant could not be assigned")
        else:
            return tenant.un

In [10]:
# initiating a new tenant and assigning
tenant = new_tenant()
new_tenant_assign(tenant)

######## BUGS ########
### 1. the second tenant is not being placed properly 
# --> check gender_check() regarding the index or values (fixed!)
### 2. place_roommate isn't functioning (fixed!)
# --> ???
### 3. 

### as of July 2, the assigning by room type, roommate preference, and gender is working properly
# + but need to adjust it to academic year system (i.e two roommates instead of one)
# + should optimize by move-in and move-out dates 
# + 

### as of July 27, working fine with a more realistic sheet
# gender_check() works accurately!
# + should update the status on code or excel (DECIDE!)
# + if the tenant has an assigned room, it gives an error
# + should check if their preferred roommate is a current tenant


# Roommate preference not working


'504B'

In [11]:
# save the workbook
workbook.save(filename = 'test02.xlsx')

In [12]:
room = "505"
roommates = []
for cell in COL_select("Room"):
    if room in str(cell.value):
        roommates.append(cell)

In [13]:
roommates

[<Cell 'Current roster'.A34>,
 <Cell 'Current roster'.A35>,
 <Cell 'Current roster'.A36>,
 <Cell 'Current roster'.A37>,
 <Cell 'Current roster'.A38>,
 <Cell 'Current roster'.A39>]

In [14]:
worksheet["A2"]

<Cell 'Current roster'.A2>

In [15]:
COL_select("Room")[1]

<Cell 'Current roster'.A2>

In [24]:
COL_select("Move-in date")[93].value

datetime.datetime(2021, 5, 26, 0, 0)

In [21]:
COL_select("Move-out date")[2].value

datetime.datetime(2020, 12, 31, 0, 0)

In [22]:
COL_select("Move-out date")[2].value - COL_select("Move-in date")[2].value

datetime.timedelta(days=112)